<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex08/Alexander_Valle/IA025_Aula8_Exerc%C3%ADcio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Alexander Valle RA230254'
print(f'Meu nome é {nome}')

Meu nome é Alexander Valle RA230254


In [ ]:
# Cargar el Drive helper y mount
from google.colab import drive

# Link y Codigo de autorizacion.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import necessary Python packages
import os
import numpy as np
import urllib.request

# set the working directory to the `earth-analytics-bootcamp` directory
# print the current working directory
os.getcwd()
os.chdir("/content/drive/MyDrive/IA025/")

#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 7, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)



O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [ ]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 17.9 MB/s 
     |████████████████████████████████| 596 kB 62.2 MB/s 
     |████████████████████████████████| 6.6 MB 61.6 MB/s 
     |████████████████████████████████| 86 kB 7.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [ ]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [ ]:
# Check which GPU we are using
!nvidia-smi

Wed May 25 16:20:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [ ]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        self.examples = []
        for text in tqdm_notebook(texts):
            token_ids = tokenize(text=text, tokenizer=tokenizer)
            if len(token_ids) < context_size + 1:
                continue
            # Compute n-grams:
            for i in range(len(token_ids) - context_size):
                input_ids = token_ids[i:i + context_size]
                target_id = token_ids[i + context_size]
                self.examples.append((input_ids, target_id)) 

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        input_ids, target_id = self.examples[idx]
        return torch.LongTensor(input_ids), target_id

## Testando se a implementação do MyDataset está correta

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/2 [00:00<?, ?it/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [ ]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

File ‘sample_brwac.txt’ already there; not retrieving.



In [ ]:
# Load datasets
context_size = 9

valid_examples = 100
test_examples = 100
texts = open('sample_brwac.txt').readlines()

print('Truncating for debugging purposes.')
texts = texts[:500]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

Truncating for debugging purposes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 406905
valid examples: 135562
test examples: 136690


### What is Attention?



$$
\alpha_i = \frac{\exp\left(f_{attn}\left(\text{key}_i, \text{query}\right)\right)}{\sum_j \exp\left(f_{attn}\left(\text{key}_j, \text{query}\right)\right)}, \hspace{5mm} \text{out} = \sum_i \alpha_i \cdot \text{value}_i
$$

Visually, we can show the attention over a sequence of words as follows:

<center width="100%" style="padding:25px"><img src="https://github.com/PyTorchLightning/lightning-tutorials/raw/main/course_UvA-DL/05-transformers-and-MH-attention/attention_example.svg" width="750px"></center>



### Scaled Dot Product Attention



$$\text{Attention}(Q,K,V)=\text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$



<center width="100%"><img src="https://github.com/PyTorchLightning/lightning-tutorials/raw/main/course_UvA-DL/05-transformers-and-MH-attention/scaled_dot_product_attn.svg" width="210px"></center>



In [ ]:
import torch.nn.functional as F
class LanguageModel(torch.nn.Module):
    def __init__(self, vocab_size, context_size, embedding_dim):
        """        Implements the Self-attention, decoder-only."
        Args:   vocab_size (int): Size of the input vocabulary.
                context_size (int): Size of the sequence to consider as context for prediction.
                embedding_dim (int): Dimension of the embedding layer for each word in the context.
        """
        super().__init__()
        self.D=embedding_dim  
        self.L=context_size  
        # These compute the queries, keys and values for all  
        self.W_keys    = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.W_queries = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.W_values  = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.Wo  = nn.Linear(embedding_dim, embedding_dim, bias=False)

        self.C= nn.Embedding(vocab_size, embedding_dim)#word embeddings
        self.P= nn.Embedding(context_size, embedding_dim)# positional_embeddings of the words
        self.softmax = nn.Softmax(dim=-1)
        #self.heads =embedding_dim//context_size
        self.ff1= nn.Linear(embedding_dim, vocab_size)#feedforward

        hsf=2#hidden size factor
        feedforward_dim=hsf*embedding_dim
        self.L1 = nn.Linear(embedding_dim, feedforward_dim)
        self.Drop =nn.Dropout(0.8)
        self.relu = nn.ReLU()#inplace=True
        self.L2 = nn.Linear(feedforward_dim, vocab_size, bias=False)
		    # Maps the final output sequence to class logits
        self.ff2 = nn.Sequential(self.L1 ,self.Drop,self.relu,self.L2)#feedforward
        self.debug=False

    def attention(self, x):
        Q = self.W_queries(x[:, -1, :]).unsqueeze(1) # shape = B, 1, D
        K = self.W_keys(x) # shape = B, C, D
        V = self.W_values(x) # shape = B, C, D
        #Attention(Q,K,V)=softmax(QKT/sqrt(dk))V
        #scores = torch.matmul(Q,torch.transpose(K,1,2))/math.sqrt(self.embedding_dim) # scores shape: (B, 1, L)
        scores = torch.matmul(Q,torch.transpose(K,1,2))/(self.D**0.5) 
        #probs = self.softmax(scores)  #
        probs = F.softmax(scores, dim=-1)
        #probs =torch.nn.functional.softmax(scores, dim=2)
        A=torch.matmul(probs, V) #   shape: (B, 1, D)
        if self.debug:
            print('x.shape:', x.shape)
            print('Q.shape:', Q.shape)
            print('K.shape:', K.shape)
            print('V.shape:', V.shape)
            print('scores.shape:', scores.shape)
            print('probs.shape:', probs.shape)
            print('Attention.shape:', A.shape)
        return A

    def forward(self, inputs):
        """
        Args:     inputs is a LongTensor of shape (batch_size, context_size)            
        Returns:  logits of shape (batch_size, vocab_size)
        """
        B = inputs.shape[0]# B: batch_size
        tokens = self.C(inputs)# generate token embeddings
		    # generate position embeddings , L: context_size        
        positions = self.P(torch.arange(0, self.L).unsqueeze(0).repeat(B,1).to(inputs.device)) # fron Larissa
        #positions = self.P.weight # from Pedro Gengo
        X = tokens + positions
        E = self.Wo(self.attention(X))
        logits = self.ff2(E.view(B,-1)) # B, V  # fron Larissa 
        #logits = self.ff2(E) # B, V
        if self.debug:
          print('batch_size',B)
          print('tokens.size()',tokens.size())
          print('positions.size()',positions.size())
          print('E.shape:', E.shape)
          print('logits.shape:', logits.shape)      
        return logits

## Teste o modelo com um exemplo

In [ ]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=16,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

torch.Size([1, 29794])

In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 1938322


## Assert da Perplexidade


In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.
    Args:   logits: a FloatTensor of shape (batch_size, vocab_size)
            target: a LongTensor of shape (batch_size,)

    Returns:  A float corresponding to the perplexity
    """
    loss = nn.functional.cross_entropy(logits, target, reduction='mean')
    return torch.exp(loss)


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              30429
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [ ]:

max_examples = 100_000_000
eval_every_steps = 10000
lr = 4e-4
compare=float('inf')


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


#save_best_model = SaveBestModel()


train_losses = []
n_examples = 0
step = 0

path='/content/drive/MyDrive/IA025/'
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))
                
                #save_best_model(last_loss, 0, model, optimizer, nn.functional.cross_entropy)

                if valid_ppl<compare:#from Larisa
                    compare=valid_ppl
                    torch.save(model, path+"model.pt")
                    with open(path+"valid_ppl_model.txt", 'w') as f:
                        f.write("%s\n" % valid_ppl)
                    f.close()

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 30410.53, valid ppl: 29822.73
10000 steps; 10240000 examples so far; train ppl: 1307.28, valid ppl: 1430.46
20000 steps; 20480000 examples so far; train ppl: 718.34, valid ppl: 1772.07
30000 steps; 30720000 examples so far; train ppl: 475.18, valid ppl: 3051.73
40000 steps; 40960000 examples so far; train ppl: 354.18, valid ppl: 5465.35
50000 steps; 51200000 examples so far; train ppl: 286.37, valid ppl: 10805.40
60000 steps; 61440000 examples so far; train ppl: 243.02, valid ppl: 20605.94
70000 steps; 71680000 examples so far; train ppl: 212.81, valid ppl: 38868.67
80000 steps; 81920000 examples so far; train ppl: 189.99, valid ppl: 77810.52
90000 steps; 92160000 examples so far; train ppl: 172.12, valid ppl: 133630.22


## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
model = LanguageModel(vocab_size=tokenizer.vocab_size,context_size=context_size,embedding_dim=128,).to(device)
model = torch.load(path+"model.pt", map_location=device)
model.to(device)

LanguageModel(
  (W_keys): Linear(in_features=128, out_features=128, bias=False)
  (W_queries): Linear(in_features=128, out_features=128, bias=False)
  (W_values): Linear(in_features=128, out_features=128, bias=False)
  (Wo): Linear(in_features=128, out_features=128, bias=False)
  (C): Embedding(29794, 128)
  (P): Embedding(9, 128)
  (softmax): Softmax(dim=-1)
  (ff1): Linear(in_features=128, out_features=29794, bias=True)
  (L1): Linear(in_features=128, out_features=256, bias=True)
  (Drop): Dropout(p=0.8, inplace=False)
  (relu): ReLU()
  (L2): Linear(in_features=256, out_features=29794, bias=False)
  (ff2): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): Dropout(p=0.8, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=29794, bias=False)
  )
)

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 1524.7670231603481


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz,
Eu gosto de comer pizza pois me faz,,
Eu gosto de comer pizza pois me faz,,,
Eu gosto de comer pizza pois me faz,,,,
Eu gosto de comer pizza pois me faz,,,,,
Eu gosto de comer pizza pois me faz,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,,,,,,,,,
